In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
API_KEY = "nKoQR9ILavB9YqC83xUDDgb5UqnDhGUx1wcOApGd"

import requests
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"
response = requests.get(url)
response

<Response [200]>

In [3]:
data = response.json()
data

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-07&end_date=2024-01-13&detailed=false&api_key=nKoQR9ILavB9YqC83xUDDgb5UqnDhGUx1wcOApGd',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2023-12-26&end_date=2024-01-01&detailed=false&api_key=nKoQR9ILavB9YqC83xUDDgb5UqnDhGUx1wcOApGd',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&detailed=false&api_key=nKoQR9ILavB9YqC83xUDDgb5UqnDhGUx1wcOApGd'},
 'element_count': 111,
 'near_earth_objects': {'2024-01-02': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/2415949?api_key=nKoQR9ILavB9YqC83xUDDgb5UqnDhGUx1wcOApGd'},
    'id': '2415949',
    'neo_reference_id': '2415949',
    'name': '415949 (2001 XY10)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=2415949',
    'absolute_magnitude_h': 19.37,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.3552670883,
      'estimated_diameter_max': 0.7944013596},


In [4]:
data.keys()

dict_keys(['links', 'element_count', 'near_earth_objects'])

In [5]:
asteroids_data = []
target = 10000
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"
response = requests.get(url)
data = response.json()
details = data['near_earth_objects']
while len(asteroids_data) < target:
    for date,ast_details in details.items():
        for ast in ast_details:
             if ast['close_approach_data']:
                approach_data = ast['close_approach_data'][0]
                miss_distance = approach_data['miss_distance']
                relative_velocity = approach_data['relative_velocity']
                asteroids_data.append(dict(
                id = int(ast['id']),
                neo_reference_id = int(ast['neo_reference_id']),
                name = ast['name'],
                magnitude = float(ast['absolute_magnitude_h']),
                dia_min = ast['estimated_diameter']['kilometers']['estimated_diameter_min'],
                dia_max = ast['estimated_diameter']['kilometers']['estimated_diameter_max'],
                hazardous = ast['is_potentially_hazardous_asteroid'],
                close_approach_date = approach_data['close_approach_date'],
                relative_velocity = float(relative_velocity['kilometers_per_hour']),
                miss_distance_ast = float(miss_distance['astronomical']),
                miss_distance_km = float(miss_distance['kilometers']),
                miss_distance_lunar = float(miss_distance['lunar']),
                orbiting_body = approach_data['orbiting_body']
                ))
                if len(asteroids_data) == target:
                      break
        if len(asteroids_data) == target:
                break
    url = data['links']['next']

In [6]:
asteroids_data[1]

{'id': 3160747,
 'neo_reference_id': 3160747,
 'name': '(2003 SR84)',
 'magnitude': 26.0,
 'dia_min': 0.0167708462,
 'dia_max': 0.0375007522,
 'hazardous': False,
 'close_approach_date': '2024-01-02',
 'relative_velocity': 38589.054833182,
 'miss_distance_ast': 0.1323425924,
 'miss_distance_km': 19798169.933318187,
 'miss_distance_lunar': 51.4812684436,
 'orbiting_body': 'Earth'}

In [7]:
len(asteroids_data)

10000

In [8]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import pymysql

In [10]:
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='Alby@0308')

In [11]:
cur = myconnection.cursor()

In [12]:
cur.execute("create database if not exists Near_earth_objects")

1

In [13]:
myconnection = pymysql.connect(host = '127.0.0.1',user='root',passwd='Alby@0308',database = "Near_earth_objects")

In [14]:
cur = myconnection.cursor()

In [36]:
cur.execute("""create table if not exists asteroids(id int,name varchar(30),absolute_magnitude_h float,estimated_diameter_min_km float,estimated_diameter_max_km float,is_potentially_hazardous_asteroid boolean)""")

0

In [37]:
cur.execute("""create table if not exists close_approach(neo_reference_id int,close_approach_date date,relative_velocity_kmph float,miss_distance_ast float,miss_distance_km float,miss_distance_lunar float,orbiting_body varchar(10))""")

0

In [17]:
asteroids_data[5]

{'id': 3591616,
 'neo_reference_id': 3591616,
 'name': '(2011 YP10)',
 'magnitude': 23.94,
 'dia_min': 0.0433066885,
 'dia_max': 0.0968366995,
 'hazardous': False,
 'close_approach_date': '2024-01-02',
 'relative_velocity': 20905.2083584121,
 'miss_distance_ast': 0.0704899554,
 'miss_distance_km': 10545147.184234997,
 'miss_distance_lunar': 27.4205926506,
 'orbiting_body': 'Earth'}

In [38]:
insert_query = '''insert into asteroids(id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid) values (%s,%s,%s,%s,%s,%s)''' 

In [39]:
for i in asteroids_data:
    values = (i["id"], i["name"], i["magnitude"], i["dia_min"], i["dia_max"], i["hazardous"])
    cur.execute(insert_query, values)
myconnection.commit()

In [40]:
insert_query_1 = ('''insert into close_approach(neo_reference_id,close_approach_date,relative_velocity_kmph,miss_distance_ast,miss_distance_km,miss_distance_lunar,orbiting_body) values (%s,%s,%s,%s,%s,%s,%s)''') 

In [41]:
for i in asteroids_data:
    values = (i["neo_reference_id"], i["close_approach_date"], i["relative_velocity"], i["miss_distance_ast"], i["miss_distance_km"], i["miss_distance_lunar"], i["orbiting_body"])
    cur.execute(insert_query_1, values)
myconnection.commit()

In [42]:
cur.execute('select*from asteroids')

10000

In [43]:
cur.execute('select*from close_approach')

10000

In [24]:
# 1 Count how many times each asteroid has approached Earth
query = '''SELECT name, count(*) AS approach_count FROM asteroids GROUP BY name'''

In [25]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,approach_count
0,415949 (2001 XY10),1001
1,(2003 SR84),1001
2,(2005 YQ96),1001
3,(2009 HC21),1001
4,(2010 XA11),1001
...,...,...
106,(2023 YV1),990
107,(2024 AM),990
108,(2024 AW),990
109,(2024 AL2),990


In [26]:
# 2  Average velocity of each asteroid over multiple approaches
query = '''SELECT a.name, AVG(CAST(c.relative_velocity_kmph AS DECIMAL(10,2))) AS avg_velocity FROM (SELECT DISTINCT id, name FROM asteroids) AS a JOIN close_approach AS c ON a.id = c.neo_reference_id GROUP BY a.name'''

In [27]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,avg_velocity
0,415949 (2001 XY10),57205.890000
1,(2003 SR84),38589.050000
2,(2005 YQ96),56413.020000
3,(2009 HC21),21891.120000
4,(2010 XA11),31468.980000
...,...,...
106,(2023 YV1),29233.780000
107,(2024 AM),32484.080000
108,(2024 AW),30585.570000
109,(2024 AL2),23938.550000


In [28]:
# 3 List top 10 fastest asteroids
query = """SELECT a.name, c.relative_velocity_kmph FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id ORDER BY c.relative_velocity_kmph DESC LIMIT 10"""

In [29]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,relative_velocity_kmph
0,(2013 NT11),136268.0
1,(2013 NT11),136268.0
2,(2013 NT11),136268.0
3,(2013 NT11),136268.0
4,(2013 NT11),136268.0
5,(2013 NT11),136268.0
6,(2013 NT11),136268.0
7,(2013 NT11),136268.0
8,(2013 NT11),136268.0
9,(2013 NT11),136268.0


In [30]:
# 4 Find potentially hazardous asteroids that have approached Earth more than 3 times
query = """SELECT a.name, COUNT(*) AS approach_count FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id WHERE a.is_potentially_hazardous_asteroid = 1 GROUP BY a.name HAVING COUNT(*) > 3"""

In [31]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,approach_count
0,(2005 YQ96),1184183
1,(2012 SD22),1184183
2,613286 (2005 YQ96),1184183
3,669051 (2012 SD22),1158300
4,450293 (2004 LV3),1158300
5,(2008 WZ94),1158300
6,(2017 YZ1),1158300
7,585310 (2017 YZ1),1158300
8,199003 (2005 WJ56),1158300
9,434188 (2003 AD23),1158300


In [44]:
# 5 Find the month with the most asteroid approaches
query = """SELECT MONTH(close_approach_date) AS approach_month, COUNT(*) AS total_approaches FROM close_approach GROUP BY MONTH(close_approach_date) ORDER BY total_approaches DESC"""

In [45]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,approach_month,total_approaches
0,1,10000


In [46]:
# 6 Get the asteroid with the fastest ever approach speed
query = """SELECT a.name, c.relative_velocity_kmph FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id ORDER BY c.relative_velocity_kmph DESC LIMIT 1"""

In [47]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,relative_velocity_kmph
0,(2013 NT11),136268.0


In [48]:
# 7 Sort asteroids by maximum estimated diameter (descending)
query = """SELECT name, estimated_diameter_max_km FROM asteroids ORDER BY estimated_diameter_max_km DESC"""

In [49]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,estimated_diameter_max_km
0,199003 (2005 WJ56),1.386880
1,199003 (2005 WJ56),1.386880
2,199003 (2005 WJ56),1.386880
3,199003 (2005 WJ56),1.386880
4,199003 (2005 WJ56),1.386880
...,...,...
9995,(2017 UJ2),0.003927
9996,(2017 UJ2),0.003927
9997,(2017 UJ2),0.003927
9998,(2017 UJ2),0.003927


In [50]:
# 8 An asteroid whose closest approach is getting nearer over time
query = """SELECT a.name, c.close_approach_date, c.miss_distance_km FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id ORDER BY a.name, c.close_approach_date"""

In [51]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,close_approach_date,miss_distance_km
0,(2002 AO11),2024-01-06,60009500.0
1,(2002 AO11),2024-01-06,60009500.0
2,(2002 AO11),2024-01-06,60009500.0
3,(2002 AO11),2024-01-06,60009500.0
4,(2002 AO11),2024-01-06,60009500.0
...,...,...,...
900905,676480 (2016 GZ215),2024-01-03,47416400.0
900906,676480 (2016 GZ215),2024-01-03,47416400.0
900907,676480 (2016 GZ215),2024-01-03,47416400.0
900908,676480 (2016 GZ215),2024-01-03,47416400.0


In [ ]:
# 9 Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.
query = """ SELECT a.name, c.close_approach_date, c.miss_distance_km FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id JOIN (SELECT neo_reference_id, MIN(miss_distance_km) AS min_miss_distance FROM close_approach GROUP BY neo_reference_id) AS sub ON c.neo_reference_id = sub.neo_reference_id AND c.miss_distance_km = sub.min_miss_distance ORDER BY c.miss_distance_km ASC LIMIT 5"""

In [52]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,close_approach_date,miss_distance_km
0,(2002 AO11),2024-01-06,60009500.0
1,(2002 AO11),2024-01-06,60009500.0
2,(2002 AO11),2024-01-06,60009500.0
3,(2002 AO11),2024-01-06,60009500.0
4,(2002 AO11),2024-01-06,60009500.0
...,...,...,...
900905,676480 (2016 GZ215),2024-01-03,47416400.0
900906,676480 (2016 GZ215),2024-01-03,47416400.0
900907,676480 (2016 GZ215),2024-01-03,47416400.0
900908,676480 (2016 GZ215),2024-01-03,47416400.0


In [53]:
# 10 List names of asteroids that approached Earth with velocity > 50,000 km/h
query = """SELECT DISTINCT a.name FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id WHERE c.relative_velocity_kmph > 50000 LIMIT 5"""

In [54]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name
0,415949 (2001 XY10)
1,(2005 YQ96)
2,(2017 YD8)
3,(2019 KK5)
4,613286 (2005 YQ96)


In [55]:
# 11 Count how many approaches happened per month
query = """SELECT MONTH(close_approach_date) AS month, COUNT(*) AS total_approaches FROM close_approach GROUP BY MONTH(close_approach_date) ORDER BY month"""

In [56]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,month,total_approaches
0,1,10000


In [57]:
# 12 Find asteroid with the highest brightness 
query = """SELECT name, absolute_magnitude_h FROM asteroids ORDER BY absolute_magnitude_h ASC LIMIT 5"""

In [58]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,absolute_magnitude_h
0,199003 (2005 WJ56),18.16
1,199003 (2005 WJ56),18.16
2,199003 (2005 WJ56),18.16
3,199003 (2005 WJ56),18.16
4,199003 (2005 WJ56),18.16


In [59]:
# 13 Get number of hazardous vs non-hazardous asteroids
query = """SELECT is_potentially_hazardous_asteroid AS hazardous_status, COUNT(*) AS total FROM asteroids GROUP BY is_potentially_hazardous_asteroid"""

In [60]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,hazardous_status,total
0,0,9007
1,1,993


In [61]:
# 14 Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.
query = """SELECT a.name, c.close_approach_date, c.miss_distance_lunar FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id WHERE c.miss_distance_lunar < 1 ORDER BY c.miss_distance_lunar ASC LIMIT 1"""

In [62]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,close_approach_date,miss_distance_lunar
0,(2024 AD),2024-01-04,0.631031


In [63]:
# 15 Find asteroids that came within 0.05 AU
quary = """SELECT a.name, c.close_approach_date, c.miss_distance_ast FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id WHERE c.miss_distance_ast < 0.05 ORDER BY c.miss_distance_ast ASC LIMIT 3"""

In [64]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,close_approach_date,miss_distance_lunar
0,(2024 AD),2024-01-04,0.631031


In [65]:
# 16 List asteroids with the largest size range
query = """SELECT name, estimated_diameter_max_km - estimated_diameter_min_km AS size_range FROM asteroids ORDER BY size_range DESC LIMIT 3"""

In [66]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,size_range
0,199003 (2005 WJ56),0.76665
1,199003 (2005 WJ56),0.76665
2,199003 (2005 WJ56),0.76665


In [79]:
# 17 Find asteroids that only ever approached once
query = """SELECT a.name, COUNT(c.neo_reference_id) AS approach_count FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id GROUP BY a.name HAVING COUNT(c.neo_reference_id) = 1"""

In [80]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,approach_count


In [69]:
# 18 Find the average estimated size of all asteroids
query = """SELECT AVG((estimated_diameter_min_km + estimated_diameter_max_km)/2) AS avg_estimated_diameter_km FROM asteroids"""

In [70]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,avg_estimated_diameter_km
0,0.141699


In [75]:
# 19 list hazaedous assteroids with diamoter estimates over 1000km
query = """SELECT name, estimated_diameter_max_km FROM asteroids WHERE is_potentially_hazardous_asteroid = TRUE AND estimated_diameter_max_km > 1000"""

In [76]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,name,estimated_diameter_max_km


In [73]:
# 20 Count how many hazardous asteroids approached earth each year
query = """SELECT YEAR(c.close_approach_date) AS approach_year, COUNT(*) AS hazardous_approaches FROM asteroids AS a JOIN close_approach AS c ON a.id = c.neo_reference_id WHERE a.is_potentially_hazardous_asteroid = TRUE GROUP BY YEAR(c.close_approach_date) ORDER BY approach_year"""

In [74]:
cur.execute(query)
result=cur.fetchall()
df=pd.DataFrame(result,columns=[i[0]for i in cur.description])
df

,approach_year,hazardous_approaches
0,2024,89643
